# Practical Part

Implement a small app that uses all concepts introduced in the theory part.

Ideas:

    - Fusion-Chef: Combines two dishes into something new - also knows the fitting cocktail to everything. Saves delicious sounding recipes in the memory.
    - QnA Bot: Searches a prefilled memory for answers to certain questions
    - Roleplay Gamemaster: Generates roleplay encounters and keeps track of the party's items & stats
    - Shell Meister: Can translate natural language into shell commands and executes them (only recommended when using a devcontainer). Saves the results in memory if the system is still working....

In [9]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding
from semantic_kernel.contents import ChatHistory
from semantic_kernel.memory.semantic_text_memory import SemanticTextMemory
from semantic_kernel.memory.volatile_memory_store import VolatileMemoryStore
from semantic_kernel.core_plugins.text_memory_plugin import TextMemoryPlugin

kernel = Kernel()

service_id = "chat-gpt"
chat_service = AzureChatCompletion(
    service_id=service_id,
)

embedding_service_id = "embeddings"
kernel.add_service(AzureTextEmbedding(service_id=embedding_service_id))

settings = chat_service.get_prompt_execution_settings_class()
settings.temperature = 0.7
settings.top_p = 0.8

kernel.add_service(chat_service)

# string with a mix from two dishes and a cocktail
chat_history_1 = "I had a great time at the restaurant. I ordered the spaghetti carbonara and the tiramisu. The espresso martini was amazing!"
chat_history_2 = "I ordered pizza and a salad. The margarita was delicious!"
chat_history_3 = "I ordered pasta and a ice. The Vodka was amazing!"


memory_store = VolatileMemoryStore()
memory = SemanticTextMemory(storage=memory_store, embeddings_generator=kernel.get_service(embedding_service_id))
kernel.add_plugin(TextMemoryPlugin(memory), "TextMemoryPlugin")


await memory.save_information(
        collection="facts", id="fact1", text=str(chat_history_1), description="First chat history"
    )


system_message = """
Du bist ein Chatbot, der zwei Gerichte zusammenfügst und einen passenden Cocktail auswählst. 
"""

chat_function = kernel.add_function(
    prompt=system_message + chat_history_1 + chat_history_2 + chat_history_3,
    function_name="chat",
    plugin_name="chat",
    settings=settings
)


response = await kernel.invoke(chat_function,user_input=input)
print(response)




Das klingt nach einem kulinarischen Erlebnis! Lass uns die Gerichte und Cocktails kombinieren:

**Gerichtskombination:** 
Wir kombinieren die **Spaghetti Carbonara** mit einer **Pizza Margherita**. Diese Kombination vereint die cremige Bolognese der Carbonara mit der frischen Tomatensauce und dem Mozzarella der Pizza. Das ergibt einen herzhaften, italienischen Genuss!

**Passender Cocktail:** 
Zu dieser Kombination empfehlen wir einen **Aperol Spritz**. Dieser erfrischende Cocktail mit seinem leicht bitteren Geschmack und der spritzigen Note passt hervorragend zu den Aromen von Pasta und Pizza und sorgt für einen sommerlichen Touch.

Lass es dir schmecken!
